# Análise da taxa de aprovação das disciplinas
O objetivo desse notebook é realizar uma análise da taxa de aprovação das disciplinas da UFRN, por período.

**Perguntas orientadoras**:
- Qual o índice de aprovação da disciplina FMC2 em 2019.1?

# Os dados
Os dados que iremos utilizar foram retirados do site de [dados abertos da UFRN](http://dados.ufrn.br/). Os pacotes de dados foram:

- [Docentes](http://dados.ufrn.br/dataset/docentes)
- [Matriculas em Componentes](http://dados.ufrn.br/dataset/matriculas-componentes)
- [Turmas](http://dados.ufrn.br/dataset/turmas)

## Dependências
Para o download desses dados, iremos utilizar o pacote [odufrn-downloader](https://github.com/odufrn/odufrn-downloader).

In [ ]:
!pip install -U --user odufrn_downloader

## Importações

In [1]:
import numpy as np
import pandas as pd
from os import listdir
from odufrn_downloader import ODUFRNDownloader

## Download dos arquivos

In [2]:
# Objeto
ufrn_data = ODUFRNDownloader()
# Processo de download
ufrn_data.download_packages([
    'docentes', 'matriculas-componentes', 'turmas',
], path='data', dictionary=False)

Baixando Docentes...
Baixando Matrículas de 2019.1...
Baixando Matrículas de 2018.2...
Baixando Matrículas de 2018.1...
Baixando Matrículas de 2017.2...
Baixando Matrículas de 2017.1...
Baixando Matrículas de 2016.2...
Baixando Matrículas de 2016.1...
Baixando Matrículas de 2015.2...
Baixando Matrículas de 2015.1...
Baixando Matrículas de 2014.2...
Baixando Matrículas de 2014.1...
Baixando Matrículas de 2013.2...
Baixando Matrículas de 2013.1...
Baixando Matrículas de 2012.2...
Baixando Matrículas de 2011.2...
Baixando Matrículas de 2012.1...
Baixando Matrículas de 2011.1...
Baixando Matrículas de 2010.2...
Baixando Matrículas de 2010.1...
Baixando Matrículas de 2009.2...
Baixando Matrículas de 2009.1...
Baixando Turmas de 2018.1...
Baixando Turmas de 2018.2...
Baixando Turmas de 2017.1...
Baixando Turmas de 2017.2...
Baixando Turmas de 2016.1...
Baixando Turmas de 2016.2...
Baixando Turmas de 2015.1...
Baixando Turmas de 2015.2...
Baixando Turmas de 2014.1...
Baixando Turmas de 2014.2

## Leitura dos arquivos
Como temos muitos arquivos, não podemos nos dar ao luxo de alocar todos ele em memória, para isso iremos especificar os campos que queremos usar de cada pacote de dados.

**Docentes**:

| Campo | Tipo | Descrição | Obrigatório? |
| ----- | ---- | --------- | ------------ |
| siape | Inteiro | Matrícula SIAPE do docente | Sim |
| nome | Texto | Nome do docente | Sim |
| formacao | Texto | Denominação da formação/titulação do docente | Não |

**Turmas**:

| Campo | Tipo | Descrição | Obrigatório? |
| ----- | ---- | --------- | ------------ |
| id_turma | Inteiro | Identificador da turma | Sim |
| siape | Inteiro | Siape do docente interno (siape - caso seja um docente servidor da instituição). | Não |
| id_componente | Inteiro | Identificador do componente curricular da turma. | Sim |
| nivel_ensino | Texto | Nível de ensino da turma. | Sim |
| situacao_turma | Texto | Situação da turma. | Sim |
| tipo | Inteiro | Identificador para informar o tipo da Turma. | Sim |

**Matrículas em Componentes**:

| Campo | Tipo | Descrição | Obrigatório? |
| ----- | ---- | --------- | ------------ |
| id_turma | Inteiro | Identificador da turma do discente. | Sim |
| id_discente | Texto | Identificador do discente da matrícula. | Sim |
| id_curso | Inteiro | Identificador do curso do discente. | Sim |
| media_final | Decimal | Média final do discente. | Sim |
| descricao | Texto | Situação da matrícula. | Sim |

In [2]:
# Campos que iremos usar de cada dataset
target_columns = {
    'docentes': [
        'siape',
        'nome',
        'formacao',
    ],
    'turmas': [
        'id_turma',
        'siape',
        'id_componente_curricular',
        'nivel_ensino',
        'situacao_turma',
        'tipo',
    ],
    'matriculas-componentes': [
        'id_turma',
        'discente',
        'id_curso',
        'media_final',
        'descricao',
    ],
    
}

Realizamos o processo de filtragem dos campos e concatenação dos DataFrames. No caso de **turmas** e **matriculas-componentes**, iremos criar o campo *ano_periodo*. 

In [4]:
# Dicionário que conterá os datasets
dfs = {
    'docentes': [],
    'turmas': [],
    'matriculas-componentes': [],
}



# Percorremos as pastas com os datasets
for package in listdir('data'):
    # Percorremos cada arquivo da pasta
    for filename in listdir('data/'+package):
        # Caminho para o arquivo
        path_file = 'data/'+package+'/'+filename
        # Lemos o dataset
        df = pd.read_csv(path_file, sep=';', usecols=target_columns[package])
        # Adiciona a coluna para identificar os anos e períodos
        if package in ['turmas', 'matriculas-componentes']:
            df['ano_periodo'] = filename[-10:-4]
        # Adicionamos o dataframe a lista de dataframe
        dfs[package].append(df)
        
# Ao final, concatenamos todos os datasets, gerando um só para cada pacote
for key in dfs.keys():
    dfs[key] = pd.concat(dfs[key], ignore_index=True)

A função **[pandas.DataFrame.info](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html)** nos fornecerá um sumário dos dados que temos. 

In [5]:
dfs['docentes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2361 entries, 0 to 2360
Data columns (total 3 columns):
siape       2361 non-null int64
nome        2361 non-null object
formacao    2361 non-null object
dtypes: int64(1), object(2)
memory usage: 55.5+ KB


In [6]:
dfs['turmas'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371632 entries, 0 to 371631
Data columns (total 7 columns):
id_turma                    371632 non-null int64
siape                       358806 non-null float64
id_componente_curricular    371632 non-null int64
nivel_ensino                371632 non-null object
tipo                        371632 non-null object
situacao_turma              371632 non-null object
ano_periodo                 371632 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 19.8+ MB


In [7]:
dfs['matriculas-componentes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079106 entries, 0 to 13079105
Data columns (total 6 columns):
id_turma       int64
discente       object
id_curso       float64
media_final    object
descricao      object
ano_periodo    object
dtypes: float64(1), int64(1), object(4)
memory usage: 598.7+ MB


# Calcular a taxa de aprovação das disciplinas
Aqui iremos calcular a taxa de aprovação das disciplinas por período.

## Filtro
Primeiro iremos realizar um filtro nas turmas, pegando apenas as que são de graduação e foram consolidadas.

In [8]:
# Filtramos as turmas de graduação e turmas consolidadas
df_turmas_filter = dfs['turmas'][
    (dfs['turmas']['nivel_ensino'] == 'GRADUAÇÃO')
    & (dfs['turmas']['situacao_turma'] == 'CONSOLIDADA')
    & (dfs['turmas']['tipo'] == 'REGULAR')
]

# Reseta os indexes
df_turmas_filter.reset_index(drop=True, inplace=True)

In [9]:
df_turmas_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264494 entries, 0 to 264493
Data columns (total 7 columns):
id_turma                    264494 non-null int64
siape                       261767 non-null float64
id_componente_curricular    264494 non-null int64
nivel_ensino                264494 non-null object
tipo                        264494 non-null object
situacao_turma              264494 non-null object
ano_periodo                 264494 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 14.1+ MB


In [10]:
df_turmas_filter.head()

,id_turma,siape,id_componente_curricular,nivel_ensino,tipo,situacao_turma,ano_periodo
0,1160176,1149368.0,21743,GRADUAÇÃO,REGULAR,CONSOLIDADA,2011.1
1,1160177,1856399.0,22926,GRADUAÇÃO,REGULAR,CONSOLIDADA,2011.1
2,1160178,6346790.0,25562,GRADUAÇÃO,REGULAR,CONSOLIDADA,2011.1
3,1160179,346468.0,25562,GRADUAÇÃO,REGULAR,CONSOLIDADA,2011.1
4,1160179,6346790.0,25562,GRADUAÇÃO,REGULAR,CONSOLIDADA,2011.1


Agora vamos realizar o filtro nas matriculas, pegando as matriculas completaram a disciplina de alguma forma.

In [31]:
# Filtramos as matriculas que completaram a disciplina de alguma forma
df_matriculas_filter = dfs['matriculas-componentes'][
    dfs['matriculas-componentes']['descricao'].isin([
            'APROVADO',
            'APROVADO POR NOTA',
            'REPROVADO',
            'REPROVADO POR FALTAS',
            'REPROVADO POR MÉDIA E POR FALTAS',
            'REPROVADO POR NOTA',
            'REPROVADO POR NOTA E FALTA'
        ])
]

# Reseta os indexes
df_matriculas_filter.reset_index(drop=True, inplace=True)

# Dropna
df_matriculas_filter = df_matriculas_filter[df_matriculas_filter['media_final'].notna()]

# Muda as virgulas para pontos
df_matriculas_filter['media_final'] = df_matriculas_filter['media_final'].str.replace(
    ',', '.'
)

# Muda do tipo da coluna
df_matriculas_filter['media_final'] = df_matriculas_filter.media_final.astype(float)

In [32]:
df_matriculas_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11149013 entries, 0 to 11237076
Data columns (total 6 columns):
id_turma       int64
discente       object
id_curso       float64
media_final    float64
descricao      object
ano_periodo    object
dtypes: float64(2), int64(1), object(3)
memory usage: 595.4+ MB


In [30]:
df_matriculas_filter.head()

,id_turma,discente,id_curso,media_final,descricao,ano_periodo
0,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
1,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
2,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
3,57508432,a9bbcaaf5d35b4f71ffd224bb1fedbe5,2000011.0,8.2,APROVADO,2009.1
4,57508432,a9bbcaaf5d35b4f71ffd224bb1fedbe5,2000011.0,8.2,APROVADO,2009.1


In [29]:
df_matriculas_filter[df_matriculas_filter.media_final.isna()]['ano_periodo'].unique()

array(['2009.1', '2014.1', '2011.2', '2010.2', '2018.1', '2011.1',
       '2014.2', '2010.1', '2017.2', '2019.1', '2012.2', '2015.1',
       '2017.1', '2016.1', '2016.2', '2013.1', '2015.2', '2012.1',
       '2009.2', '2018.2', '2013.2'], dtype=object)

In [21]:
dfs['matriculas-componentes']['media_final'].isna().sum()

1677719

In [24]:
dfs['matriculas-componentes'][dfs['matriculas-componentes']['media_final'].isna()].head()

,id_turma,discente,id_curso,media_final,descricao,ano_periodo
0,57508432,fad2f949bdce831a13d3380055ee07ec,2000011.0,NaN,INDEFERIDO,2009.1
1,57508432,d8221e79084eea45367dc26f59bd3adf,2000011.0,NaN,INDEFERIDO,2009.1
5,57508432,91580e1dd90ed3811c753915c7b3596c,2000002.0,NaN,INDEFERIDO,2009.1
6,57508432,fad2f949bdce831a13d3380055ee07ec,2000011.0,NaN,INDEFERIDO,2009.1
10,57508432,17b5d47ecd1c29d07b68d4c954593732,2000011.0,NaN,INDEFERIDO,2009.1


In [23]:
df_matriculas_filter.media_final.unique()

array([ nan,  5. ,  4. , 10. ,  9.7,  8. ,  7. ,  0. ,  9. ,  6.3,  9.3,
        6. ,  6.7,  7.7,  3. ,  1. ,  6.4,  7.3,  6.6,  7.9,  8.5,  8.6,
        5.8,  9.1,  2.2,  7.5,  6.1,  9.2,  8.8,  9.5,  8.2,  2.8,  8.3,
        6.8,  0.7,  5.6,  9.4,  0.3,  7.4,  5.1,  7.1,  6.9,  8.1,  5.4,
        6.2,  2. ,  5.9,  8.7,  7.2,  9.8,  6.5,  9.9,  1.5,  7.6,  8.4,
        7.8,  8.9,  5.3,  5.2,  5.7,  0.2,  5.5,  2.3,  0.9,  1.3,  0.6,
        2.1,  2.7,  1.2,  2.4,  1.1,  1.9,  9.6,  3.5,  3.4,  2.9,  3.3,
        3.9,  3.6,  2.5,  2.6,  3.8,  0.8,  1.8,  3.2,  0.4,  0.5,  4.2,
        4.5,  1.7,  1.4,  1.6,  4.9,  4.4,  3.7,  4.6,  3.1,  4.3,  4.1,
        4.7,  4.8,  0.1])

## Agrupamentos
Aqui iremos agrupar os datasets, para podermos realizar operações mais interessantes. Como o dataset referente a matriculas é muito grande para gerarmos cardinalidade com agrupamentos (podendo ocasionar um `MemoryError`), iremos primeiro calcular as médias de cada discentes para depois só utilizarmos essas médias.

In [49]:
# Calcula os calculos de média dos discentes
df_matriculas_group = df_matriculas_filter.groupby(['id_turma', 'discente'])

In [50]:
df_matriculas_group.head()

,id_turma,discente,id_curso,media_final,descricao,ano_periodo
0,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
1,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
2,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
3,57508432,a9bbcaaf5d35b4f71ffd224bb1fedbe5,2000011.0,8.2,APROVADO,2009.1
4,57508432,a9bbcaaf5d35b4f71ffd224bb1fedbe5,2000011.0,8.2,APROVADO,2009.1
...,...,...,...,...,...,...
11237072,1213594,2cd3cd2590ba78cf33d0d773d4fc0cae,4970.0,9,APROVADO,2013.2
11237073,1213594,2cd3cd2590ba78cf33d0d773d4fc0cae,4970.0,9,APROVADO,2013.2
11237074,1219446,a52b7f0f65e6a42148820a9178a26169,9513330.0,5,APROVADO,2013.2
11237075,1219446,e30c37f19cdeb8108c19a78ac408d322,9300592.0,5,APROVADO,2013.2


In [51]:
df_matriculas_group['media_final'].mean()

DataError: No numeric types to aggregate

In [19]:
'bc3f5fa0b73af682c8e8f5ec1663028b' == 'bc3f5fa0b73af682c8e8f5ec1663028b'

True